In [15]:
def group_bounding_boxes_per_frame(labels_file, output_file, iou_threshold=0.5):
    """
    Groups overlapping bounding boxes of the same class into a single bounding box per frame.
    Uses one of the track IDs from the grouped set for the resulting bounding box.

    Parameters:
    labels_file (str): Path to the labels.txt file containing bounding box information.
    output_file (str): Path to save the updated labels file with grouped bounding boxes.
    iou_threshold (float): IoU threshold to consider two boxes as overlapping.
    """
    def calculate_iou(box1, box2):
        # Calculate intersection coordinates
        x1_inter = max(box1[0], box2[0])
        y1_inter = max(box1[1], box2[1])
        x2_inter = min(box1[2], box2[2])
        y2_inter = min(box1[3], box2[3])

        # Calculate intersection area
        inter_area = max(0, x2_inter - x1_inter) * max(0, y2_inter - y1_inter)

        # Calculate union area
        box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
        box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
        union_area = box1_area + box2_area - inter_area

        # Return IoU
        return inter_area / union_area if union_area > 0 else 0

    # Read labels file and extract data
    with open(labels_file, 'r') as file:
        labels = [line.strip().split() for line in file]

    # Group by frame
    frames = {}
    for line in labels:
        frame = int(line[0])
        bbox = list(map(float, line[6:10]))
        obj_data = line[:6] + [bbox] + line[10:]
        if frame not in frames:
            frames[frame] = []
        frames[frame].append(obj_data)

    # Process each frame for grouping
    updated_labels = []
    for frame, objects in frames.items():
        grouped = []
        used = set()

        for i, obj1 in enumerate(objects):
            if i in used:
                continue

            # Start a new group
            group = [obj1]
            used.add(i)

            for j, obj2 in enumerate(objects):
                if j in used or obj1[2] != obj2[2]:  # Skip if already grouped or different class
                    continue

                # Calculate IoU
                bbox1 = obj1[6]
                bbox2 = obj2[6]
                iou = calculate_iou(bbox1, bbox2)

                if iou >= iou_threshold:
                    group.append(obj2)
                    used.add(j)

            grouped.append(group)

        # Merge grouped bounding boxes and update labels
        for group in grouped:
            if len(group) == 1:
                # Single box, no grouping needed
                updated_labels.append(" ".join(map(str, group[0][:6] + group[0][6] + group[0][7:])))
            else:
                # Merge bounding boxes
                min_x = min(obj[6][0] for obj in group)
                min_y = min(obj[6][1] for obj in group)
                max_x = max(obj[6][2] for obj in group)
                max_y = max(obj[6][3] for obj in group)
                merged_bbox = [min_x, min_y, max_x, max_y]

                # Use the first object's data for the new bounding box
                representative = group[0]
                merged_obj = representative[:6] + merged_bbox + representative[10:]
                updated_labels.append(" ".join(map(str, merged_obj)))

    # Write updated labels to output file
    with open(output_file, 'w') as file:
        file.writelines(label + "\n" for label in updated_labels)

    print(f"Grouped bounding boxes saved to {output_file}")


In [14]:
group_bounding_boxes_per_frame(
    labels_file=r'C:\Users\User\Documents\GitHub\pfas_finalproject\Sequence Labels\Seq_02.txt',
    output_file=r'C:\Users\User\Documents\GitHub\pfas_finalproject\kalman\labels_grouped.txt',
    iou_threshold=0.3  # Specify IoU threshold
)


Grouped bounding boxes saved to C:\Users\User\Documents\GitHub\pfas_finalproject\kalman\labels_grouped.txt


# Remove range no depth

In [1]:
def create_labels_with_removed_x_range(labels_file, x_range, new_labels_file):
    """
    Creates a new labels file with track IDs removed if their bounding box falls within a specified x-coordinate range.

    Parameters:
    labels_file (str): Path to the original labels file.
    x_range (tuple): A tuple (x_min, x_max) specifying the range of x-coordinates.
    new_labels_file (str): Path to save the new labels file with track IDs removed.

    Returns:
    None
    """
    x_min, x_max = x_range

    # Read original labels
    with open(labels_file, 'r') as file:
        labels = file.readlines()

    # Prepare modified data
    modified_labels = []
    for line in labels:
        columns = line.strip().split()
        bbox = list(map(float, columns[6:10]))  # Extract bounding box: [left, top, right, bottom]
        left, _, right, _ = bbox

        # Check if the bounding box overlaps with the specified x range
        if x_min <= left <= x_max or x_min <= right <= x_max:
            continue  # Skip this line if the bounding box falls in the x range

        # Add line to modified labels if it doesn't match the exclusion criteria
        modified_labels.append(line)

    # Write modified labels to new file
    with open(new_labels_file, 'w') as file:
        file.writelines(modified_labels)

    print(f"New labels file created: {new_labels_file}")


In [1]:
create_labels_with_removed_x_range(
    labels_file=r'C:\Users\User\Documents\dataset\pfas\34759_final_project_rect\seq_02\labels.txt',
    x_range=(660, 910),
    new_labels_file=r'C:\Users\User\Documents\GitHub\pfas_finalproject\kalman\artificial_occlusion_better_02.txt'
)

NameError: name 'create_labels_with_removed_x_range' is not defined

# Remove range depth

In [1]:
def create_labels_with_removed_x_range(labels_file, x_range, new_labels_file):
    """
    Creates a new labels file with track IDs removed if their bounding box falls within a specified x-coordinate range.

    Parameters:
    labels_file (str): Path to the original labels file.
    x_range (tuple): A tuple (x_min, x_max) specifying the range of x-coordinates.
    new_labels_file (str): Path to save the new labels file with track IDs removed.

    Returns:
    None
    """
    x_min, x_max = x_range

    # Read original labels
    with open(labels_file, 'r') as file:
        labels = file.readlines()

    # Prepare modified data
    modified_labels = []
    for line in labels:
        columns = line.strip().split()
        
        # Parse bounding box coordinates
        bbox_left = float(columns[3])  # bbox_left
        bbox_right = float(columns[5])  # bbox_right

        # Check if the bounding box overlaps with the specified x range
        if x_min <= bbox_left <= x_max or x_min <= bbox_right <= x_max:
            continue  # Skip this line if the bounding box falls in the x range

        # Add line to modified labels if it doesn't match the exclusion criteria
        modified_labels.append(line)

    # Write modified labels to new file
    with open(new_labels_file, 'w') as file:
        file.writelines(modified_labels)

    print(f"New labels file created: {new_labels_file}")


In [7]:
labels_file = r'C:\Users\User\Documents\GitHub\pfas_finalproject\kalman_3d\depth_fixed_02.txt'
new_labels_file = r'C:\Users\User\Documents\GitHub\pfas_finalproject\kalman_3d\depth_fixed_occlusion_02.txt'
x_range = (660, 910)  # Specify the range of x-coordinates to exclude

create_labels_with_removed_x_range(labels_file, x_range, new_labels_file)

New labels file created: C:\Users\User\Documents\GitHub\pfas_finalproject\kalman_3d\depth_fixed_occlusion_02.txt


In [5]:
def visualize_labels_with_classes(labels_file_path, images_folder_path, delay=100):
    """
    Visualizes bounding boxes from YOLO labels with different colors for classes and IDs.

    Args:
        labels_file_path (str): Path to the YOLO labels file.
        images_folder_path (str): Path to the folder containing images.
        delay (int): Time in milliseconds to display each frame. Default is 100 ms.

    Returns:
        None
    """
    import cv2
    import os
    import pandas as pd

    # Define column names for the YOLO labels file
    headers = [
        "frame", "track id", "type", "truncated", "occluded", "alpha",
        "bbox_left", "bbox_top", "bbox_right", "bbox_bottom",
        "dimensions_height", "dimensions_width", "dimensions_length",
        "location_x", "location_y", "location_z",
        "rotation_y", "score"
    ]

    # Load the labels file into a DataFrame
    try:
        df = pd.read_csv(labels_file_path, sep=' ', names=headers)
    except Exception as e:
        print(f"Error reading labels file: {e}")
        return

    # Define a mapping of object types to colors
    classification_colors = {
        "Pedestrian": (34, 139, 34),  # Green
        "Car": (0, 0, 128),           # Navy Blue
        "Cyclist": (139, 0, 0),       # Dark Red
        "misc": (128, 0, 128)         # Purple (default for unclassified objects)
    }

    # Sort frames to process in order
    unique_frames = sorted(df["frame"].unique())

    for frame in unique_frames:

        # Get bounding box data for the current frame
        frame_data = df[df["frame"] == frame]

        # Construct the image path
        image_path = os.path.join(images_folder_path, f"{frame:010d}.png")
        if not os.path.exists(image_path):
            print(f"Image not found: {image_path}")
            continue

        # Read the image
        image = cv2.imread(image_path)
        if image is None:
            print(f"Failed to load image: {image_path}")
            continue

        # Draw bounding boxes on the image
        for _, row in frame_data.iterrows():
            bbox = [row["bbox_left"], row["bbox_top"], row["bbox_right"], row["bbox_bottom"]]
            track_id = row["track id"]
            object_type = row["type"]

            # Determine the color based on the object type
            color = classification_colors.get(object_type, (128, 128, 128))  # Default to gray if type is unknown

            # Draw the bounding box
            cv2.rectangle(
                image,
                (int(bbox[0]), int(bbox[1])),
                (int(bbox[2]), int(bbox[3])),
                color,
                2
            )

            # Add a label for the ID and type
            label = f"ID: {track_id}"
            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 0.5
            font_thickness = 1
            text_size, _ = cv2.getTextSize(label, font, font_scale, font_thickness)
            text_width, text_height = text_size

            # Background rectangle for text
            cv2.rectangle(
                image,
                (int(bbox[0]), int(bbox[1]) - text_height - 5),
                (int(bbox[0]) + text_width, int(bbox[1])),
                (0, 0, 0),  # Black background
                -1
            )
            # Draw the label text
            cv2.putText(
                image,
                label,
                (int(bbox[0]), int(bbox[1]) - 5),
                font,
                font_scale,
                (255, 255, 255),  # White text
                font_thickness
            )

        # Add frame number at the top-left corner
        frame_text = f"Frame: {frame}"
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 1
        font_thickness = 2
        text_color = (255, 255, 255)  # White text
        bg_color = (0, 0, 0)  # Black background

        # Get text size
        (text_width, text_height), baseline = cv2.getTextSize(frame_text, font, font_scale, font_thickness)

        # Set position for text and background
        text_x = 10
        text_y = 30
        bg_x1 = text_x - 5
        bg_y1 = text_y - text_height - 5
        bg_x2 = text_x + text_width + 5
        bg_y2 = text_y + baseline

        # Draw the background rectangle
        cv2.rectangle(image, (bg_x1, bg_y1), (bg_x2, bg_y2), bg_color, -1)

        # Put the text on top of the background
        cv2.putText(image, frame_text, (text_x, text_y), font, font_scale, text_color, font_thickness)

        # Display the image
        cv2.imshow("YOLO Labels Visualization", image)

        # Wait for the specified delay or exit on 'q' key
        key = cv2.waitKey(delay) & 0xFF
        if key == ord('q'):
            break

    # Close all OpenCV windows
    cv2.destroyAllWindows()


In [6]:
labels_file_path = r'C:\Users\User\Documents\GitHub\pfas_finalproject\kalman_3d\depth_fixed_02.txt'
images_folder_path = r'C:\Users\User\Documents\GitHub\pfas_finalproject\kalman_3d\depth_fixed_occlusions_02.txt'

visualize_labels_with_classes(labels_file_path, images_folder_path)

Image not found: C:\Users\User\Documents\GitHub\pfas_finalproject\kalman_3d\depth_fixed_occlusions_02.txt\0000000000.png
Image not found: C:\Users\User\Documents\GitHub\pfas_finalproject\kalman_3d\depth_fixed_occlusions_02.txt\0000000001.png
Image not found: C:\Users\User\Documents\GitHub\pfas_finalproject\kalman_3d\depth_fixed_occlusions_02.txt\0000000002.png
Image not found: C:\Users\User\Documents\GitHub\pfas_finalproject\kalman_3d\depth_fixed_occlusions_02.txt\0000000003.png
Image not found: C:\Users\User\Documents\GitHub\pfas_finalproject\kalman_3d\depth_fixed_occlusions_02.txt\0000000004.png
Image not found: C:\Users\User\Documents\GitHub\pfas_finalproject\kalman_3d\depth_fixed_occlusions_02.txt\0000000005.png
Image not found: C:\Users\User\Documents\GitHub\pfas_finalproject\kalman_3d\depth_fixed_occlusions_02.txt\0000000006.png
Image not found: C:\Users\User\Documents\GitHub\pfas_finalproject\kalman_3d\depth_fixed_occlusions_02.txt\0000000007.png
Image not found: C:\Users\User\D

In [2]:
def filter_short_lived_consecutive_track_ids(labels_file, output_file, min_consecutive_frames=10):
    """
    Filters out track IDs that appear in fewer consecutive frames than the specified threshold.

    Parameters:
    labels_file (str): Path to the input labels.txt file.
    output_file (str): Path to save the filtered labels file.
    min_consecutive_frames (int): Minimum number of consecutive frames a track ID must appear in to be retained.

    Returns:
    None
    """
    import pandas as pd

    # Define column names for the labels file
    headers = [
        "frame", "track_id", "type", "truncated", "occluded", "alpha",
        "bbox_left", "bbox_top", "bbox_right", "bbox_bottom",
        "dimensions_height", "dimensions_width", "dimensions_length",
        "location_x", "location_y", "location_z",
        "rotation_y", "score"
    ]

    # Read the labels file into a DataFrame
    try:
        df = pd.read_csv(labels_file, sep=' ', names=headers)
    except Exception as e:
        print(f"Error reading labels file: {e}")
        return

    # Function to find maximum consecutive frames for a track
    def max_consecutive_frames(frames):
        if len(frames) < min_consecutive_frames:
            return 0
        
        # Sort frames
        sorted_frames = sorted(frames)
        
        # Track consecutive frames
        max_consecutive = 1
        current_consecutive = 1
        
        for i in range(1, len(sorted_frames)):
            if sorted_frames[i] == sorted_frames[i-1] + 1:
                current_consecutive += 1
                max_consecutive = max(max_consecutive, current_consecutive)
            else:
                current_consecutive = 1
        
        return max_consecutive

    # Group by track ID and find consecutive frames
    track_consecutive_frames = df.groupby("track_id")["frame"].apply(max_consecutive_frames)
    
    # Get valid track IDs (those with sufficient consecutive frames)
    valid_track_ids = track_consecutive_frames[track_consecutive_frames >= min_consecutive_frames].index.tolist()

    # Filter the DataFrame to include only valid track IDs
    filtered_df = df[df["track_id"].isin(valid_track_ids)]

    # Save the filtered DataFrame to the output file
    try:
        filtered_df.to_csv(output_file, sep=' ', index=False, header=False)
        
        # Print some statistics
        total_tracks = len(df["track_id"].unique())
        filtered_tracks = len(filtered_df["track_id"].unique())
        print(f"Total unique track IDs: {total_tracks}")
        print(f"Filtered track IDs (with at least {min_consecutive_frames} consecutive frames): {filtered_tracks}")
        print(f"Filtered labels saved to {output_file}")
    except Exception as e:
        print(f"Error writing filtered labels file: {e}")

In [8]:
labels_file = r'C:\Users\User\Documents\GitHub\pfas_finalproject\Sequence Labels\Seq_03.txt'
output_file = r'C:\Users\User\Documents\GitHub\pfas_finalproject\kalman\filtered_labels_03.txt'

images_folder_path = r'C:\Users\User\Documents\dataset\pfas\34759_final_project_rect\seq_03\image_02\data'

filter_short_lived_consecutive_track_ids(labels_file, output_file, min_consecutive_frames=15)

# Visualize the filtered labels
visualize_labels_with_classes(output_file, images_folder_path)


Total unique track IDs: 33
Filtered track IDs (with at least 15 consecutive frames): 18
Filtered labels saved to C:\Users\User\Documents\GitHub\pfas_finalproject\kalman\filtered_labels_03.txt


# Fix depth